# Replay in Aversive Environments - Sequenceness analysis

#### _This is a template that will be parameterised and run via [Papermill](http://papermill.readthedocs.io/) for each subject_

This notebook uses the classifer trained on the localiser data to detect spontaneous state reactivation during the planning and rest phases of the task.

Analysis steps:

1. Loading task data and classifier
2. Applying the classifer to the task data to generate time X state reactivation probabilities matrices
3. Running the GLM-based sequenceness estimation procedure

## Imports

In [ ]:
import sys
sys.path.insert(0, 'code')
print(sys.executable)
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict
from sklearn.externals import joblib
import os
import papermill as pm
from state_prediction import *
from sequenceness import *
from utils import *
%matplotlib inline

np.random.seed(100)

## Parameters

In [ ]:
session_id = 'MG05513'  # ID of the scanning session
output_dir = 'data/derivatives'  # Where the output data should go
eye_tracking = True  # If True, eye-tracking measures will be used for exclusion of blink-related ICA components
n_stim = 8  # Number of stimuli, including null
shifts = [-5, 6]  # Additional timepoints to use as features
max_lag = 40  # Maximum lag to use in sequencess analysis

## State detection

### Load the classifier

First we load the classifier that we previously trained on the localiser data

In [ ]:
clf = joblib.load(os.path.join(output_dir, 'classifier', 'sub-{0}_classifier.pkl').format(session_id)) 

### Get the task data

We're interested in the planning and rest phases so we'll select these.

In [ ]:
task_epochs = mne.read_epochs(os.path.join(output_dir, 'preprocessing/task', 'sub-{0}_ses-01_task-AversiveLearningReplay_run-task_proc_ICA-epo.fif.gz').format(session_id))
planning_epochs = task_epochs['planning']
rest_epochs = task_epochs['rest']

# Get the data as a numpy array, excluding non-MEG channels
planning_X = planning_epochs.get_data()[:, 29:302, :] # MEG signals: n_epochs, n_channels, n_times
rest_X = rest_epochs.get_data()[:, 29:302, :]

In [ ]:
assert np.isnan(planning_X).any() == False, "Nans present in planning data"
assert np.isnan(rest_X).any() == False, "Nans present in rest data"
assert np.isinf(planning_X).any() == False, "Infs present in planning data"
assert np.isinf(rest_X).any() == False, "Infs present in rest data"

### State detection

Here we iterate over trials, reshape the data for each trial into the format `[n_trials, n_sensors, n_timepoints]`, where the first dimension is 1 and the final dimension is the timepoint of interest plus additional adjacent timepoints used as extra features, and finally and use the `predict_proba` method of the fitted classifier to get predicted state reactivation probabilities for every timepoint within the trial.


This involves a lot of for loops and could probably be made far more efficient...

In [ ]:
planning_state_reactivation = predict_states(planning_X, clf)

assert np.isnan(planning_state_reactivation).any() == False, "Nans present in planning state reactivation array"
assert np.isinf(planning_state_reactivation).any() == False, "Infs present in planning state reactivation array"

rest_state_reactivation = predict_states(rest_X, clf)
assert np.isnan(rest_state_reactivation).any() == False, "Nans present in rest state reactivation array"
assert np.isinf(rest_state_reactivation).any() == False, "Infs present in rest state reactivation array"

# Save state probabilities
np.save(os.path.join(output_dir, 'state_reactivation_arrays', 'planning', 'sub-{0}_planning_state_reactivation'.format(session_id)), planning_state_reactivation)
np.save(os.path.join(output_dir, 'state_reactivation_arrays', 'rest', 'sub-{0}_rest_state_reactivation'.format(session_id)), rest_state_reactivation)

In [ ]:
# Convert to StateReactivation class
planning_state_reactivation = StateReactivation(planning_state_reactivation)
rest_state_reactivation = StateReactivation(rest_state_reactivation)

### Plot state detection probabilities

We can plot these state X time arrays to view how states are reactivated over time on each trial.

_This is commented out here as it's an interactive plot which takes a while to run/load and increases the size of the notebook_

In [ ]:
# # REST PHASE
# plot_state_prob(rest_state_probabilities, 'Rest phase')

# # PLANNING PHASE
# plot_state_prob(planning_state_probabilities, 'Planning phase')

## Sequenceness analysis

After determining the state reactivation probabilities for each trial, we can submit this data to the sequenceness analysis. We use a GLM approach here.

### Load transition matrix

Here we load the transition matrix of the task, which is necessary for sequenceness analysis. We then subset this matrix to get the two branches of the task tree.

In [ ]:
transition_matrix = np.loadtxt(r'task/Task_information/transition_matrix.txt')

matrices = [transition_matrix]

for n, i in enumerate([5, 6]):
    matrices.append(select_path(transition_matrix, i))

### Run the analysis for each trial

This returns a dictionary with entries representing forwards and reverse sequenceness, along with the difference between the two.

This is repeated for 1000 permuted transition matrices that share no transitions with the true matrix or its inverse.

In [ ]:
permuted_matrices = generate_permuted_matrices(transition_matrix, n_permutations=1000)

In [ ]:
rest_sequenceness, null_rest_sequenceness = rest_state_reactivation.get_sequenceness(max_lag, matrices, alpha=True, remove_first=True, permuted_matrices=permuted_matrices)
planning_sequenceness, null_planning_sequenceness = planning_state_reactivation.get_sequenceness(max_lag, matrices, alpha=True, remove_first=True, permuted_matrices=permuted_matrices)


In [ ]:
f, ax = plt.subplots(2, figsize=(6, 10), facecolor='white')

labels = ['Whole matrix', 'Arm 1', 'Arm 2']

for i in range(3):
    ax[0].plot(rest_sequenceness['difference'][..., i].mean(axis=0), label=labels[i])
ax[0].set_xlabel('Lag')
ax[0].set_title('Rest sequenceness')
ax[0].set_ylabel(r'Backward $\leftarrow$ Sequenceness $\rightarrow$ Forward')
ax[0].legend()

for i in range(3):
    ax[1].plot(planning_sequenceness['difference'][..., i].mean(axis=0), label=labels[i])
ax[1].set_xlabel('Lag')
ax[1].set_title('Planning sequenceness')
ax[1].set_ylabel(r'Backward $\leftarrow$ Sequenceness $\rightarrow$ Forward')
ax[1].legend();



## Save the sequenceness data

In [ ]:
joblib.dump(planning_sequenceness, os.path.join(output_dir, 'sequenceness', 'planning', 'sub-{0}_planning_sequenceness.pkl'.format(session_id)))
joblib.dump(rest_sequenceness, os.path.join(output_dir, 'sequenceness', 'rest', 'sub-{0}_rest_sequenceness.pkl'.format(session_id)))
joblib.dump(null_planning_sequenceness, os.path.join(output_dir, 'sequenceness', 'planning', 'sub-{0}_null_planning_sequenceness.pkl'.format(session_id)))
joblib.dump(null_rest_sequenceness, os.path.join(output_dir, 'sequenceness', 'rest', 'sub-{0}_null_rest_sequenceness.pkl'.format(session_id)))